In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import warnings
import re

In [ ]:
import multiprocessing

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

In [ ]:
pip install pytorch-tabnet;

In [ ]:
pip install optuna;

In [ ]:
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold,cross_val_score
from sklearn.linear_model import LassoCV,LassoLarsIC,Lasso,LinearRegression

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def lasso_moran_select(data,s):

    X_coords = data.iloc[:,:-3]

    y = data['y']

    X1 = data.X1.values
    X2 = data.X2.values

    moran = data.iloc[:,2:-3].values

    moran_X1_names = ["moran_X1_" + str(i) for i in range(moran.shape[1])]
    X_coords.loc[:,moran_X1_names] = moran*X1.reshape(-1,1)

    moran_X2_names = ["moran_X2_" + str(i) for i in range(moran.shape[1])]
    X_coords.loc[:,moran_X2_names] = moran*X2.reshape(-1,1)

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    if s == 'mse':
        model = LassoCV(cv=kf,  alphas=np.logspace(-4, -2, 20))
        model.fit(X_coords, y)
        r2_scores = cross_val_score(model, X_coords, y, cv=kf, scoring='r2')

        print("MSE LASSO CV R2 score: ", np.mean(r2_scores))

    if s == 'bic':
        model = LassoLarsIC(criterion='bic')
        model.fit(X_coords, y)

        r2_scores = cross_val_score(model, X_coords, y, cv=kf, scoring='r2')

        print("BIC LASSO CV R2 score: ", np.mean(r2_scores))


    selected_index = np.where(model.coef_ != 0)[0]
    selected_feature = list(X_coords.columns[selected_index])

    moran_X1s = [s for s in selected_feature if s.startswith('moran_X1_')]
    moran_X2s = [s for s in selected_feature if s.startswith('moran_X2_')]
    moran_s = [s for s in selected_feature if re.match(r"^moran_\d+$", s)]

    numbers_s = set(int(x.split('_')[1]) for x in moran_s)
    numbers_x1 = set(int(x.split('_')[2]) for x in moran_X1s)
    numbers_x2 = set(int(x.split('_')[2]) for x in moran_X2s)

    # Find intersection of numbers in both lists
    common_numbers = numbers_s.union(numbers_x1.union(numbers_x2))

    moran_selected = ['moran_' + str(num) for num in sorted(common_numbers)]

    return moran_selected

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/MEM Colab Runs')

In [ ]:
from tabnet_model import TabNet_fit

In [ ]:
weights = ['queen','exp'] #2
models = ['tabnet'] #1
selection = ['xy','all','mse','bic'] #4

In [ ]:
warnings.filterwarnings('ignore')

for w in weights:
    data = pd.read_csv('/content/drive/MyDrive/MEM Colab Runs/data/grid_moran_' + w + '.csv',index_col=0)

    for s in selection:

        if s == 'all':
            selected = list(data.columns[2:-3])
        if s == 'xy':
            selected = list(data.columns[-3: -1])
        if s =='mse':
            selected = lasso_moran_select(data,s)
        if s == 'bic':
            selected = lasso_moran_select(data,s)
        if s == 'aic':
            selected = lasso_moran_select(data,s)

        X_coords = data[['X1','X2']  + selected]

        moran = X_coords[selected]
        moran = (np.array(moran) - np.array(moran).mean(axis=0))/np.array(moran).std(axis=0)
        X_coords[selected] = moran

        y = data['y']

        for m in models:

            model,best_loss = TabNet_fit(X_coords.values, y.values.reshape(-1,1), timeout=60*30)

            #model.fit(X_coords, y, estimator_list = [m], **settings)

            name = "/content/drive/MyDrive/MEM Colab Runs/models/grid_" + m + "_moran_" + s + "_lasso_" + w + ".model"

            pickle.dump(model, open(name, 'wb'))

            print(name + ' CV-R2 = {0:.3g}'.format(best_loss))

[I 2024-05-10 19:46:44,422] A new study created in memory with name: no-name-49e70716-dc02-426e-b197-5733532f8cfc


Stop training because you reached max_epochs = 369 with best_epoch = 358 and best_val_0_rmse = 0.77328

Early stopping occurred at epoch 309 with best_epoch = 237 and best_val_0_rmse = 0.7196
Stop training because you reached max_epochs = 369 with best_epoch = 325 and best_val_0_rmse = 0.79927
Stop training because you reached max_epochs = 369 with best_epoch = 347 and best_val_0_rmse = 0.77465


[I 2024-05-10 19:49:37,133] Trial 0 finished with value: -0.9477249292925418 and parameters: {'mask_type': 'sparsemax', 'n_da': 64, 'n_steps': 3, 'gamma': 1.1, 'n_shared': 1, 'lambda_sparse': 1.951722464144948e-08, 'patienceScheduler': 14, 'patience': 72, 'epochs': 369}. Best is trial 0 with value: -0.9477249292925418.


Stop training because you reached max_epochs = 369 with best_epoch = 360 and best_val_0_rmse = 0.80028
Stop training because you reached max_epochs = 181 with best_epoch = 180 and best_val_0_rmse = 1.04726
Stop training because you reached max_epochs = 181 with best_epoch = 165 and best_val_0_rmse = 0.87299
Stop training because you reached max_epochs = 181 with best_epoch = 180 and best_val_0_rmse = 1.08027
Stop training because you reached max_epochs = 181 with best_epoch = 180 and best_val_0_rmse = 0.94582


[I 2024-05-10 19:50:45,889] Trial 1 finished with value: -0.9149979064242191 and parameters: {'mask_type': 'sparsemax', 'n_da': 64, 'n_steps': 2, 'gamma': 1.1, 'n_shared': 1, 'lambda_sparse': 3.3205591037519537e-07, 'patienceScheduler': 9, 'patience': 60, 'epochs': 181}. Best is trial 0 with value: -0.9477249292925418.


...


Early stopping occurred at epoch 257 with best_epoch = 157 and best_val_0_rmse = 0.9452

Early stopping occurred at epoch 360 with best_epoch = 269 and best_val_0_rmse = 0.9369

Early stopping occurred at epoch 337 with best_epoch = 246 and best_val_0_rmse = 1.14677

Early stopping occurred at epoch 248 with best_epoch = 157 and best_val_0_rmse = 1.12654

Early stopping occurred at epoch 293 with best_epoch = 202 and best_val_0_rmse = 1.00237


[I 2024-05-11 00:03:10,233] Trial 12 finished with value: -0.9085939967887123 and parameters: {'mask_type': 'entmax', 'n_da': 60, 'n_steps': 2, 'gamma': 1.2, 'n_shared': 2, 'lambda_sparse': 4.791526679772836e-05, 'patienceScheduler': 12, 'patience': 91, 'epochs': 497}. Best is trial 6 with value: -0.9321225098015617.



Early stopping occurred at epoch 301 with best_epoch = 210 and best_val_0_rmse = 0.8729

Early stopping occurred at epoch 257 with best_epoch = 171 and best_val_0_rmse = 0.92339

Early stopping occurred at epoch 277 with best_epoch = 191 and best_val_0_rmse = 0.94298

Early stopping occurred at epoch 204 with best_epoch = 118 and best_val_0_rmse = 1.07575
Stop training because you reached max_epochs = 340 with best_epoch = 316 and best_val_0_rmse = 0.99348


[I 2024-05-11 00:04:59,333] Trial 13 finished with value: -0.9213061899631892 and parameters: {'mask_type': 'entmax', 'n_da': 60, 'n_steps': 1, 'gamma': 1.0, 'n_shared': 3, 'lambda_sparse': 1.984329576941398e-05, 'patienceScheduler': 12, 'patience': 86, 'epochs': 340}. Best is trial 6 with value: -0.9321225098015617.



Early stopping occurred at epoch 250 with best_epoch = 164 and best_val_0_rmse = 0.79547
/content/drive/MyDrive/MEM Colab Runs/models/grid_tabnet_moran_bic_lasso_exp.model CV-R2 = 0.932


### LASSO Results

In [ ]:
for w in weights:
    data = pd.read_csv('/content/drive/MyDrive/MEM Colab Runs/data/grid_moran_' + w + '.csv',index_col=0)

    for s in ['mse','bic','aic']:

        if s =='mse':
            selected = lasso_moran_select(data,s)
            print(len(selected))
        if s == 'bic':
            selected = lasso_moran_select(data,s)
            print(len(selected))

MSE LASSO CV R2 score:  0.7952897584979441
172
BIC LASSO CV R2 score:  0.7724836600621674
64
MSE LASSO CV R2 score:  0.8328460914833382
90
BIC LASSO CV R2 score:  0.8254597912608508
47


### Linear Model

In [ ]:
X_coords = data.iloc[:,:2]

model = LinearRegression()

# Set up cross-validation strategy
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and calculate R2 scores
r2_scores = cross_val_score(model, X_coords, y, cv=kf, scoring='r2')

In [ ]:
r2_scores.mean()

0.4645777849249237